In [1]:
import pandas as pd
from openpyxl import load_workbook
import os

In [4]:
def split_excel(source, export, split_columns, sheet_name,name_columns=[], p_first_line=2):
        
    # Chargement du fichier source dans DataFrame
    if p_first_line>2:
        df =  pd.read_excel(source,skiprows=p_first_line-2)
    else:
        df =  pd.read_excel(source)
   
    df.loc[df[split_columns].isna()==True, split_columns] ="Vide" # Valeur par défaut pour vide
    #df = df.sort_values(split_columns)
    df = df.reset_index(drop=True)
    
    # Ajout d'un champ index utilisé par la suite pour déterminer les intervales de découpe
    df["Index"] = df.index
    # Création d'un DataFrame de calcul intermediaire pour stocker les paramètres de découpe
    df_prep = pd.DataFrame()
    
    if len(name_columns) == 0:
        name_columns = split_columns
        df_prep = df[name_columns]
    else:
        if (split_columns in name_columns)==False:
            to_add = name_columns
            to_add.append(split_columns)
            df_prep=df[to_add]
        else:
            df_prep = df[name_columns]
    
    df_prep = df_prep.drop_duplicates()
    
    # Calcul des bornes
    df_min = df.groupby(by=[split_columns])["Index"].min().reset_index()
    df_min = df_min.rename(columns={"Index":"Min"})
    df_max = df.groupby(by=[split_columns])["Index"].max().reset_index()
    df_max = df_max.rename(columns={"Index":"Max"})
    
    df_prep = df_prep.merge(df_min, on=[split_columns], how="left")
    df_prep = df_prep.merge(df_max, on=[split_columns], how="left")
    df_prep["Min"] = df_prep["Min"].apply(lambda x: x+p_first_line)
    df_prep["Max"] = df_prep["Max"].apply(lambda x: x+p_first_line)
    
    df_prep.to_excel(r'C:\Users\sgasmi\Desktop\Export Python\Test.xlsx')
    # Détermination des lignes à supprimer à la fin
    
    for i, val in enumerate(df_prep.iterrows()):
    
        df_prep.at[i, "Line_to_delete_end"] = len(df)-df_prep.at[i, "Max"]
    
    # Loop de suppression des lignes et export
    
    for i, val in enumerate(df_prep.iterrows()):
        print(df_prep.at[i,name_columns[0]])
        exp_df = load_workbook(source)
        sh = exp_df[sheet_name]
        
        compl_dest = ""
        
        if len(name_columns)>1:
            for val in name_columns:
                compl_dest= compl_dest+ str(df_prep.at[i,val]) + "_"
            compl_dest = compl_dest[:-1]
        else:
            compl_dest= str(df_prep.at[i,split_columns])
            
        dest_file_folder = export + "\\" + compl_dest

        print(dest_file_folder)

        if os.path.exists(dest_file_folder)== False:
            try:
                os.mkdir(dest_file_folder)
            except:
                pass
        if i==0:
            sh.delete_rows(df_prep.at[i, "Max"]+1, sh.max_row + 1)
        else:
            sh.delete_rows(df_prep.at[i, "Max"]+1, sh.max_row + 1)
            sh.delete_rows(p_first_line, df_prep.at[i, "Min"]-p_first_line)
        
        for i in range(3, int(sh.max_row+1)):
           
            sh['P'+str(i)] = '=IF($O'+ str(i) + '<>"",VLOOKUP($O' + str(i) + ',REFUO,2,0),"")'
            sh['Q'+str(i)] = '=IF($O'+ str(i) + '<>"",VLOOKUP($O' + str(i) + ',REFUO,14,0),"")'
            sh['R'+str(i)] = '=IF($O'+ str(i) + '<>"",VLOOKUP($O' + str(i) + ',REFUO,15,0),"")'
            sh['S'+str(i)] = '=IF($O'+ str(i) + '<>"",VLOOKUP($O' + str(i) + ',REFUO,17,0),"")'
            sh['T'+str(i)] = '=IF($O'+ str(i) + '<>"",VLOOKUP($O' + str(i) + ',REFUO,18,0),"")'
            sh['U'+str(i)] = '=IF($O'+ str(i) + '<>"",VLOOKUP($O' + str(i) + ',REFUO,19,0),"")'
            sh['V'+str(i)] = '=IF($O'+ str(i) + '<>"",VLOOKUP($O' + str(i) + ',REFUO,21,0),"")'
            
        #sh.protection.enable()
        dest_file_folder=dest_file_folder + "\\" + compl_dest +  ".xlsx"
        exp_df.save(dest_file_folder)

    
    #slit_excel(source)

In [5]:
#split_excel(source=r"C:\Users\sgasmi\Desktop\RRH\Fichier contrôle UO.xlsx", export=r"C:\Users\sgasmi\Desktop\RRH\Export", split_columns="BU",sheet_name="Effectif", name_columns=["Nom d'usage RESP RH actuel", "Prénom RESP RH actuel"], p_first_line=3)
split_excel(source=r"C:\Users\sgasmi\Desktop\RRH\Fichier contrôle UO.xlsx", export=r"C:\Users\sgasmi\Desktop\RRH\Export", split_columns="BU",sheet_name="Effectif", name_columns=["BU"], p_first_line=3)

CORP-FRA
C:\Users\sgasmi\Desktop\RRH\Export\CORP-FRA
DPIT
C:\Users\sgasmi\Desktop\RRH\Export\DPIT
DTI
C:\Users\sgasmi\Desktop\RRH\Export\DTI
FL
C:\Users\sgasmi\Desktop\RRH\Export\FL
FRAMATOME
C:\Users\sgasmi\Desktop\RRH\Export\FRAMATOME
IB
C:\Users\sgasmi\Desktop\RRH\Export\IB
NP
C:\Users\sgasmi\Desktop\RRH\Export\NP
OMCA
C:\Users\sgasmi\Desktop\RRH\Export\OMCA
PCM
C:\Users\sgasmi\Desktop\RRH\Export\PCM
SPIC
C:\Users\sgasmi\Desktop\RRH\Export\SPIC


In [15]:
folder = r"C:\Users\sgasmi\Desktop\RRH\Export"   
sources = pd.DataFrame(columns=["Fichier", "Folder", "File Date", "Conservé"])
i = 0

for root, dirs, files in os.walk(folder):
    for filename in files:
        sources.at[i,"Fichier"] = filename
        sources.at[i, "Folder"] = folder + "\\" + filename[:-5]
        sources.at[i, "File Date"] = filename[10:][:8]
        print(sources.at[i, "File Date"])
        try:
            sources.at[i, "File Date"] = dt.datetime.strptime(sources.at[i, "File Date"], '%d%m%Y').date()
        except:
            sources.at[i, "Conservé"] = "False"
        i+=1

lsx



xlsx







In [16]:
print(sources)

          Fichier                                        Folder File Date  \
0   CORP-FRA.xlsx   C:\Users\sgasmi\Desktop\RRH\Export\CORP-FRA       lsx   
1       DPIT.xlsx       C:\Users\sgasmi\Desktop\RRH\Export\DPIT             
2        DTI.xlsx        C:\Users\sgasmi\Desktop\RRH\Export\DTI             
3         FL.xlsx         C:\Users\sgasmi\Desktop\RRH\Export\FL             
4  FRAMATOME.xlsx  C:\Users\sgasmi\Desktop\RRH\Export\FRAMATOME      xlsx   
5         IB.xlsx         C:\Users\sgasmi\Desktop\RRH\Export\IB             
6         NP.xlsx         C:\Users\sgasmi\Desktop\RRH\Export\NP             
7       OMCA.xlsx       C:\Users\sgasmi\Desktop\RRH\Export\OMCA             
8        PCM.xlsx        C:\Users\sgasmi\Desktop\RRH\Export\PCM             
9       SPIC.xlsx       C:\Users\sgasmi\Desktop\RRH\Export\SPIC             

  Conservé  
0    False  
1    False  
2    False  
3    False  
4    False  
5    False  
6    False  
7    False  
8    False  
9    False  


In [34]:
for i, val in enumerate(sources.iterrows()):
    source_f= sources.at[i, "Folder"] + "\\"+ str(sources.at[i, "Fichier"])
    print(source)                   
    split_excel(source=source_f, export=sources.at[i, "Folder"], split_columns="Nom d'usage RESP RH actuel",sheet_name="Effectif", name_columns=["Nom d'usage RESP RH actuel", "Prénom RESP RH actuel"], p_first_line=3)
#split_excel(source=r"C:\Users\sgasmi\Desktop\RRH\Fichier contrôle UO.xlsx", export=r"C:\Users\sgasmi\Desktop\RRH\Export", split_columns="BU",sheet_name="Effectif", name_columns=["BU"], p_first_line=3)

C:\Users\sgasmi\Desktop\RRH\Export\SPIC\SPIC.xlsx
BESSAA
C:\Users\sgasmi\Desktop\RRH\Export\CORP-FRA\BESSAA_JASMINE
BESSON
C:\Users\sgasmi\Desktop\RRH\Export\CORP-FRA\BESSON_SEBASTIEN
DARD
C:\Users\sgasmi\Desktop\RRH\Export\CORP-FRA\DARD_JEAN FRANCOIS
GROUT
C:\Users\sgasmi\Desktop\RRH\Export\CORP-FRA\GROUT_CAMILLE
MINARD
C:\Users\sgasmi\Desktop\RRH\Export\CORP-FRA\MINARD_JEAN-LUC
SENGIER
C:\Users\sgasmi\Desktop\RRH\Export\CORP-FRA\SENGIER_AURELIA
C:\Users\sgasmi\Desktop\RRH\Export\SPIC\SPIC.xlsx
SENGIER
C:\Users\sgasmi\Desktop\RRH\Export\DPIT\SENGIER_AURELIA
C:\Users\sgasmi\Desktop\RRH\Export\SPIC\SPIC.xlsx
BIOTEAU
C:\Users\sgasmi\Desktop\RRH\Export\DTI\BIOTEAU_CAROLINE
DEPOIL
C:\Users\sgasmi\Desktop\RRH\Export\DTI\DEPOIL_JULIEN
GROUT
C:\Users\sgasmi\Desktop\RRH\Export\DTI\GROUT_CAMILLE
LOMBARDI
C:\Users\sgasmi\Desktop\RRH\Export\DTI\LOMBARDI_BRUNO
SALAH KORICHI
C:\Users\sgasmi\Desktop\RRH\Export\DTI\SALAH KORICHI_HOUYEM
Vide
C:\Users\sgasmi\Desktop\RRH\Export\DTI\Vide_nan
C:\Users\sga